### Notebook to perform Zernike polynominal analysis of SKA1-LOW and LOFAR calibration

In [ ]:
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
#pylab.rcParams['image.cmap'] = 'rainbow' 

from telopt import *
import subprocess
print "Working from Git repository %s" % subprocess.check_output(["git",  "describe", "--long", "--all"])

def DR(J, r0=14.0, B=80, tobs=10000.*3600.0, tiono=10.0):
    return 3.397 * math.pow(J, math.sqrt(3)/4.0) * math.pow(B/r0, -5.0/6.0) * math.sqrt(tobs/tiono)

def trials(ntrials,config,name,rmin=0,rmax=200.0,tiono=10,tobs=10000*3600.0,snrthreshold=5.0):
    tp=TelPiercings()

    random.seed(781490893)

    # Minimum SNR=10 per pierce point
    DRave=0
    Jave=0.0
    for trial in range(ntrials):
        ts=TelSources()
        ts.construct(nsources=nsources, radius=wavelength/stationdiameter)


        tp.construct(ts,config,hiono=hiono,rmin=rmin,rmax=HWZ)
        if trial==0:
            tp.plot(rcore=HWZ, rmax=2.0*HWZ)

        s=numpy.sqrt(tp.assess(nnoll=nnoll, rmax=HWZ, doplot=(trial==0)))
        J=len(s[s>snrthreshold])
        if trial==0:
            Save=s/float(ntrials)
        else:
            Save=Save+s/float(ntrials)
        Jave=Jave+J/float(ntrials)
        DRave=DRave+DR(J,tobs=tobs)/float(ntrials)

    return {'dr':DRave, 's':Save, 'J':Jave}
            
def plotstats(stats):
    plt.clf()
    colors={'LOWBD2':'b', 'LOWBD2-CORE':'g', 'LOWBD2-HALO':'r', 'LOWBD2-RASTERHALO':'c', 'LOWBD2-RASTER':'m', 'LOFAR':'k'}
    for name in stats.keys():
        plt.semilogy(range(len(stats[name]['s'])), stats[name]['s'], color=colors[name], label=name)
    plt.xlabel('Singular vector index')
    plt.ylabel('Singular value')
    plt.legend(loc='upper right')
    plt.title('Singular values for %d source(s)' % nsources)
    maxs=nsources*nnoll
    plt.axes().set_xlim([1,nnoll])
    plt.axes().set_ylim([-10.0,30.0])
    name='SingularValues_sources%d' % nsources
    plt.show()
    plt.savefig('%s.pdf' % name)


ntrials=10
rhalo=40.0
nsources=7
nnoll=400
wavelength=3.0
stationdiameter=35.0
rcore=0.5
rmin=0.0
rmax=40.0
hiono=300.0
HWZ=hiono*wavelength/stationdiameter # Half width to Zero
print "FOV (HWZ) at ionosphere=%.1f km" % HWZ

### Look at all arrays

In [ ]:
for nsources in [7]:
    ts=TelSources()
    random.seed(781490893)
    ts.construct(nsources=nsources, radius=wavelength/stationdiameter)
    stats={}
    lowbd=TelArray()
    lowbd.readKML(kmlfile='V4Drsst512red_2.kml')
    lowbd.saveCSV('LOWBD2.csv')
    lowbd.plot()
    stats['LOWBD2']=trials(ntrials, lowbd, 'LOWBD2')
    stats['LOWBD2']['MST'] = lowbd.mst()
    plotstats(stats)
    lowcore=TelArray()
    lowcore.readCSV('LOWBD2-CORE', l1def='LOWBD2-CORE.csv',recenter=True)
    lowcore.plot()
    stats['LOWBD2-CORE']=trials(ntrials, lowcore, 'LOWCORE')
    stats['LOWBD2-CORE']['MST'] = lowcore.mst()
    plotstats(stats)
    lowhalo=TelArray()
    lowhalo.readCSV('LOWBD2-HALO', l1def='LOWBD2-HALO.csv')
    lowhalo.plot()
    stats['LOWBD2-HALO']=trials(ntrials, lowhalo, 'LOWBD2-HALO')
    stats['LOWBD2-HALO']['MST'] = lowhalo.mst()
    plotstats(stats)
    rmin=0.5
    lowhalo=TelArray()
    lowhalo.rasterBoolardy(nhalo=346, name='LOWRASTERHALO', nstations=346, scale=1.0, rhalo=rhalo
                       , rcore=0.0)
    lowcore=TelArray()
    lowcore.readCSV('LOWCORE', l1def='LOWBD2-CORE.csv',recenter=True)
    lowraster=TelArray().add(lowhalo,lowcore,name='LOWBD2-RASTERHALO')
    lowraster.plot()
    stats['LOWBD2-RASTERHALO']=trials(ntrials, lowraster, 'LOWBD2-RASTERHALO')
    stats['LOWBD2-RASTERHALO']['MST'] = lowraster.mst()
    plotstats(stats)
    lofar=TelArray()
    lofar.readLOFAR()
    lofar.plot()
    stats['LOFAR']=trials(ntrials, lofar, 'LOFAR')
    stats['LOFAR']['MST'] = lofar.mst()
    plotstats(stats)
    print "Number of sources %d " % nsources
    for key in stats.keys():
        print "Array %s peak=%.1f, DR=%.1f, MST=%.1f" % (key, stats[key]['s'][0], stats[key]['dr'], stats[key]['MST'])